In [1]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [3]:
import os 
from dotenv import load_dotenv 
from langchain_groq import ChatGroq

load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

os.environ['HF_TOKEN'] = os.getenv("HUGGINGFACE_TOKEN")
llm = ChatGroq(groq_api_key=groq_api_key, model="Llama3-8b-8192")
llm


ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000029E6B4E2F80>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000029E6B50C430>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

d:\Code\Python\langchain-app\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Vector store 

from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(documents, embedding=embeddings)
vectorstore

In [6]:
vectorstore.similarity_search("cat")

[Document(id='5eecd6d9-fc27-4936-b237-721c190e3451', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='8de80f3a-a55d-48f8-aadb-1dac59c23e30', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='1f3f9820-6cfe-4199-9642-7bd5ec01ae1a', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(id='60414214-c702-4c46-97b9-7c67bf75eba0', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [7]:
await vectorstore.asimilarity_search("cat")

[Document(id='5eecd6d9-fc27-4936-b237-721c190e3451', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='8de80f3a-a55d-48f8-aadb-1dac59c23e30', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='1f3f9820-6cfe-4199-9642-7bd5ec01ae1a', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(id='60414214-c702-4c46-97b9-7c67bf75eba0', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [8]:
vectorstore.similarity_search_with_score("cat")

[(Document(id='5eecd6d9-fc27-4936-b237-721c190e3451', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.9351056218147278),
 (Document(id='8de80f3a-a55d-48f8-aadb-1dac59c23e30', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  1.5740898847579956),
 (Document(id='1f3f9820-6cfe-4199-9642-7bd5ec01ae1a', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
  1.595690369606018),
 (Document(id='60414214-c702-4c46-97b9-7c67bf75eba0', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
  1.665792465209961)]

In [11]:
retriever = vectorstore.as_retriever(
    search_type="similarity", 
    search_kwargs={"k": 1}
)
retriever.batch(["cat", "dog"])

[[Document(id='5eecd6d9-fc27-4936-b237-721c190e3451', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='8de80f3a-a55d-48f8-aadb-1dac59c23e30', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [12]:
# RAG 
from langchain_core.prompts import ChatPromptTemplate   
from langchain_core.runnables import RunnablePassthrough

message = """
    You are a helpful assistant. Answer the question based on the context provided.
    Context: {context}
    Question: {question}
    Answer:
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain = { "context": retriever, "question": RunnablePassthrough() } | prompt | llm 
response = rag_chain.invoke("Tell me about dogs")
print(response.content)


Based on the provided context, here's an answer to your question:

Dogs are great companions, known for their loyalty and friendliness.
